# Итоговый вариант 🌄

In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import random 
from copy import deepcopy 




from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef as MCC_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score

from catboost import Pool, cv
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

from collections import Counter 

from tqdm import tqdm

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import phik

In [3]:
data = pd.read_csv('train.csv', low_memory=False)
data

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2022-11-01,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
1,2022-11-01,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7616803.0,7616803.0,7616803.0,NaN,NaN,NaN,7616803.0,7616803.0,7616803.0,0.256261
2,2022-05-01,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261
3,2022-09-01,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.258682
4,2022-08-01,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14451,2022-07-01,1241,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.276902
14452,2022-09-01,1969,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260516
14453,2022-02-01,7116,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260005
14454,2021-08-01,7117,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261


In [4]:
set_of_good_features = set(['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294'])

In [5]:
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

our_models_2 = []

need_columns = ['target','client_id','col1454','report_date'] # столбцы нужные для удаления 

# Определите целевую переменную и группировочную переменную (client_id)
X = data.drop(columns=need_columns).fillna(-404)
y = data['target']
groups = data['client_id']

# Создайте объект StratifiedGroupKFold и задайте количество фолдов
n_splits = 5  # Задайте нужное количество фолдов
sgkf = StratifiedGroupKFold(n_splits=n_splits)

# Проходите по каждому фолду
for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y, groups)):
    # Далее вы можете использовать train_idx и val_idx для разделения данных на обучающий и валидационный фолд
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]
    
    
    
    
    
    X_train = train_data[list(set_of_good_features)]
    y_train = train_data['target']
    
    X_test = val_data[list(set_of_good_features)]
    y_test = val_data['target']
    
    train_pool = Pool(data=X_train, label=y_train)
    test_pool = Pool(data=X_test, label=y_test)

    # Создайте объект модели CatBoostClassifier
    model = CatBoostClassifier(iterations= 100,
          depth= 6,
          loss_function='Logloss', auto_class_weights="Balanced",
        boosting_type='Ordered', l2_leaf_reg= 3,
          eval_metric='AUC',
          learning_rate= 0.01,
            custom_metric = ['MCC','Accuracy','Precision','Recall','F1','NormalizedGini']
          ) 

    # Обучите модель на обучающем Pool
    model.fit(train_pool, eval_set=test_pool,verbose= False,
        plot=True,
         use_best_model= True
            )
    

    our_models_2 += [model]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [6]:
tst = pd.read_csv('test.csv', low_memory=False, sep=';')
tst

,report_date,client_id,col1,col2,col3,col4,col5,col6,col7,col8,...,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,id
0,2023-04-01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4941670.0,4941670.0,NaN,NaN,NaN,4941670.0,4941670.0,4941670.0,0.256919,0
1,2023-05-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11789500.0,11789500.0,NaN,NaN,NaN,11789500.0,11789500.0,11789500.0,0.267221,1
2,2023-05-01,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7000000.0,7000000.0,NaN,NaN,NaN,7000000.0,7000000.0,7000000.0,0.257217,2
3,2023-03-01,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3
4,2023-03-01,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10391000.0,10391000.0,10391000.0,0.257348,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3500000.0,3500000.0,3500000.0,0.256261,3637
3638,2023-04-01,8298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3638
3639,2023-03-01,835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256919,3639
3640,2023-03-01,8300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,3640


In [7]:
# Определите целевую переменную и группировочную переменную (client_id)
X = tst[['col2366',
 'col2388',
 'col2340',
 'col2382',
 'col2360',
 'col2220',
 'col2663',
 'col2294']].fillna(-404)
X

,col2366,col2388,col2340,col2382,col2360,col2220,col2663,col2294
0,1.0,30.09,5794341.03,30.09,1.0,6564363.20,0.256919,5.000000
1,2.0,63.51,13981124.40,63.51,2.0,13693099.02,0.267221,3.333333
2,2.0,49.92,11139747.84,49.92,2.0,12417006.76,0.257217,21.000000
3,2.0,37.60,10795110.40,-404.00,-404.0,10785110.30,0.256261,5.000000
4,1.0,37.05,9756932.25,-404.00,-404.0,9649462.80,0.257348,14.000000
...,...,...,...,...,...,...,...,...
3637,1.0,22.73,3906514.18,-404.00,-404.0,4318743.41,0.256261,2.000000
3638,2.0,56.90,16130694.80,-404.00,-404.0,15778680.67,0.256261,7.000000
3639,1.0,30.50,12149858.00,-404.00,-404.0,12619544.28,0.256919,14.000000
3640,2.0,54.63,11431108.98,54.63,2.0,14027694.75,0.257217,3.000000


In [8]:
preds = our_models_2[0].predict(X,prediction_type='Probability')
mmm = np.array(list(map(lambda x: x[1],preds)))
for i in range(1,5):
    preds = our_models_2[i].predict(X,prediction_type='Probability')
    mmm += np.array(list(map(lambda x: x[1],preds)))
    print(np.array(list(map(lambda x: x[1],preds)))[:10])
target = mmm / 5

[0.41656101 0.60220013 0.54023027 0.43568663 0.45078057 0.44109197
 0.51360426 0.52503987 0.38433523 0.52944275]
[0.48832382 0.52114127 0.51168904 0.47960792 0.48225373 0.48335554
 0.51003411 0.50577209 0.47734844 0.50669506]
[0.36772064 0.69277589 0.59798352 0.33750725 0.415128   0.35266652
 0.46952283 0.46761705 0.2706913  0.5202789 ]
[0.39775899 0.66727839 0.56759853 0.3802106  0.41304492 0.41570211
 0.55891935 0.40338971 0.24575763 0.56818783]


In [9]:
tst['target'] = target
tst

,report_date,client_id,col1,col2,col3,col4,col5,col6,col7,col8,...,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663,id,target
0,2023-04-01,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4941670.0,NaN,NaN,NaN,4941670.0,4941670.0,4941670.0,0.256919,0,0.414031
1,2023-05-01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11789500.0,NaN,NaN,NaN,11789500.0,11789500.0,11789500.0,0.267221,1,0.621229
2,2023-05-01,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7000000.0,NaN,NaN,NaN,7000000.0,7000000.0,7000000.0,0.257217,2,0.557368
3,2023-03-01,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3,0.405897
4,2023-03-01,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10391000.0,10391000.0,10391000.0,0.257348,4,0.442971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3500000.0,3500000.0,3500000.0,0.256261,3637,0.333952
3638,2023-04-01,8298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256261,3638,0.438876
3639,2023-03-01,835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.256919,3639,0.433530
3640,2023-03-01,8300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257217,3640,0.527654


In [10]:
tst[['id','target']].to_csv('sub.csv',sep=';',index=False)